# Scrapping LinkedIn profiles

In [430]:
import os
import requests
import re
import pandas as pd
from requests.exceptions import RequestException
import json
from pandas import json_normalize
import datetime

# Function that scrapes LinkedIn profiles

In [686]:
def scrape_linkedin_profile2(linkedin_profile_url: str):
    api_key = "2ZjHkun0Sq6KGa_o-4p7jg"
    header_dic = {"Authorization": f"Bearer {api_key}"}
    api_endpoint = "https://nubela.co/proxycurl/api/v2/linkedin"

    response = requests.get(
        api_endpoint, params={"url": linkedin_profile_url}, headers=header_dic
    )

    try:
        data = response.json()
        data = {
            k: v
            for k, v in data.items()
            if v not in ([], "", "", None)
            and k
            not in [
                "people_also_viewed",
                "recommendations",
                "similarly_named_profiles",
                "articles",
                "background_cover_image_url",
                "activities",
                "volunteer_work",
            ]
        }
        if data.get("groups"):
            for group_dict in data.get("groups"):
                group_dict.pop("profile_pic_url")

        return data

    except json.JSONDecodeError:

        return None



# Function that scrapes company's profiles

In [779]:
def scrape_linkedin_company_data(linkedin_company_url):
    api_key = "2ZjHkun0Sq6KGa_o-4p7jg"
    headers = {'Authorization': f'Bearer {api_key}'}

    api_endpoint_1 = 'https://nubela.co/proxycurl/api/linkedin/company'
    params_1 = {
        'url': linkedin_company_url,
        'use_cache': 'if-present',
    }
    response_1 = requests.get(api_endpoint_1, params=params_1, headers=headers)

    if response_1.status_code == 200:
        data = response_1.json()
        return data
    
    else:
        
        api_endpoint_2 = 'https://nubela.co/proxycurl/api/linkedin/school'
        params_2 = {
            'url': linkedin_company_url,  # Utiliza la misma URL
            'use_cache': 'if-present',
        }
        response_2 = requests.get(api_endpoint_2, params=params_2, headers=headers)

        if response_2.status_code == 200:
            data = response_2.json()
            return data
        else:
            print(f"Error al obtener datos de {linkedin_company_url} con la segunda opción.")
            return None

# Scraping and cleaning our dataframes

## First dataframe: 2015-2016 class

In [845]:
df1 = pd.read_csv('/Users/davidfernandez/Desktop/clean/scrapped/2015-2016.csv') 

df1 = df1[df1['Link perfil'] != 'Link no existente']

df1 = df1.reset_index()

df1 = df1.drop('index', axis=1)

'''We open our CSV, containing the full name and URL of the profile of our students, and we remove all those
rows that don't contain the URL of the student, because our Selenium code couldn't scrap it'''

"We open our CSV, containing the full name and URL of the profile of our students, and we remove all those\nrows that don't contain the URL of the student, because our Selenium code couldn't scrap it"

In [846]:
df1["Datos escrapeados"] = df1["Link perfil"].apply(scrape_linkedin_profile2)

# We apply our function to the row of our dataframe that contains all the URLs of our students


In [848]:
valor_a_buscar = {'code': 404, 'description': 'Person profile does not exist', 'name': 'Not Found'}


filas_filtradas = df1.loc[df1['Datos escrapeados'] == valor_a_buscar]

# We identify those students whose data we couldn't scrap

indices_a_eliminar = filas_filtradas.index

df1 = df1.drop(indices_a_eliminar)

# And we take them out 


In [849]:
df1 = df1.reset_index()

df1 = df1.drop('index', axis=1)

# We reset indexes and delete the column 'index'

In [850]:
normalized_data = pd.json_normalize(df1['Datos escrapeados']) 

# We create one column per key in our 'Datos escrapeados' column


In [851]:
df1 = pd.concat([df1, normalized_data], axis=1) # We apply those columns to our df1 dataframe


In [854]:
columnas_a_eliminar = ['Nombre', 'Primer apellido', 'Segundo apellido', 'Email universitario', 'Email personal', 'Datos escrapeados', 'public_identifier', 'profile_pic_url', 'first_name', 'last_name', 'full_name', 'headline', 'country', 'languages', 'education', 'occupation', 'connections', 'country_full_name', 'follower_count', 'summary', 'state', 'accomplishment_honors_awards', 'accomplishment_courses', 'accomplishment_projects', 'groups', 'accomplishment_publications', 'certifications', 'accomplishment_organisations', 'accomplishment_test_scores']


df1 = df1.drop(columns=columnas_a_eliminar)


In [856]:
df1 = df1.explode('experiences')

# We explode our lists in 'experiences' column, to have as many rows per student as experiences she/he has

In [857]:
df1 = df1.reset_index()

df1 = df1.drop('index', axis=1)

# We reset our indexes, to make it more clear

In [858]:
normalized_data1 = pd.json_normalize(df1['experiences']) 


df1 = pd.concat([df1, normalized_data1], axis=1) # We apply those columns to our df1 dataframe


In [860]:
df1.columns

Index(['Grado', 'Nombre completo', 'Link perfil', 'city', 'experiences',
       'company', 'company_linkedin_profile_url', 'title', 'description',
       'location', 'logo_url', 'starts_at.day', 'starts_at.month',
       'starts_at.year', 'ends_at.day', 'ends_at.month', 'ends_at.year',
       'ends_at', 'starts_at'],
      dtype='object')

In [861]:
cols_a_eliminar = ['description', 'logo_url', 'ends_at', 'starts_at', 'experiences']                  
                   
df1 = df1.drop(cols_a_eliminar, axis=1)

# We get rid of unwanted columns

In [867]:
copi1 = df1.copy() # We create a copy of our dataframe just in case


In [869]:
copi1["Datos empresa"] = copi1["company_linkedin_profile_url"].apply(scrape_linkedin_company_data)

# We scrape all the data from each company


Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de nan con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de Non

Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/640443/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/5266317/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/9295280/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/10424531/ con la segu

Error al obtener datos de https://www.linkedin.com/company/64861959/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/15092653/ con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error 

Error al obtener datos de https://www.linkedin.com/company/42942070/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/740718/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/740718/ con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de https://es.linkedin.com/company/born-living-yoga con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/3317295/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/28449229/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/3764/ con 

In [882]:
copii1 = copi1.copy()

In [883]:
normalized_data1 = pd.json_normalize(copii1['Datos empresa']) 


copii1 = pd.concat([copii1, normalized_data1], axis=1) # We apply those columns to our df1 dataframe


In [884]:
to_drop1 = ['linkedin_internal_id', 'specialities', 'Datos empresa', 'company_type', 'description', 'website', 'company_size', 'company_size_on_linkedin', 'founded_year', 'locations', 'name', 'tagline', 'universal_name_id', 'profile_pic_url', 'background_cover_image_url', 'search_id', 'similar_companies', 'affiliated_companies', 'updates', 'follower_count', 'acquisitions', 'exit_data', 'extra', 'funding_data', 'categories', 'hq.country', 'hq.city', 'hq.postal_code', 'hq.line_1', 'hq.is_hq', 'hq.state', 'hq']


copii1.drop(to_drop1, axis=1, inplace=True)



In [891]:
copii1['ends_at.day'].fillna('actualidad', inplace=True)
copii1['ends_at.month'].fillna('actualidad', inplace=True)
copii1['ends_at.year'].fillna('actualidad', inplace=True)

# If any of our ending columns have nulls, it means that that person is still working there

In [892]:
copii1.dropna(subset=['starts_at.day', 'starts_at.month', 'starts_at.year'], inplace=True)

# We get rid of all our experiences whose start date is null

In [893]:

# Combines the columns of start date and end date into 'end date'
copii1['Fecha inicio'] = copii1.apply(lambda x: pd.to_datetime(f"{int(x['starts_at.year'])}-{int(x['starts_at.month'])}-01"), axis=1)
copii1['Fecha fin'] = copii1.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else pd.to_datetime(f"{int(x['ends_at.year'])}-{int(x['ends_at.month'])}-01"), axis=1)
copii1['Duración (meses)'] = copii1.apply(lambda x: (datetime.datetime.now() - x['Fecha inicio']).days // 30 if x['ends_at.year'] == 'actualidad' else (x['Fecha fin'] - x['Fecha inicio']).days // 30, axis=1)
copii1['Fecha fin'] = copii1.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else x['Fecha fin'], axis=1)
copii1.drop(['location', 'starts_at.year', 'starts_at.day', 'starts_at.month', 'ends_at.day', 'ends_at.month', 'ends_at.year'], axis=1, inplace=True)


In [895]:
copii1.rename(columns={'company': 'Empresa', 'title': 'Puesto', 'city': 'Ciudad', 'industry': 'Sector'}, inplace=True)

# We rename some columns in spanish

In [897]:
new_order = ['Grado', 'Nombre completo', 'Link perfil', 'Puesto', 'Empresa', 'Sector', 'Ciudad', 'Fecha inicio', 'Fecha fin', 'Duración (meses)']

copii1 = copii1[new_order]

# We finally change the orders of some columns to make them more comprehensible


In [899]:
df1 = copii1.reset_index()

df1 = df1.drop('index', axis=1)

In [902]:
df1.to_csv('/Users/davidfernandez/Desktop/clean/experience/2015-2016.csv', index=False) 


## Second dataframe: 2016-2017 class

In [905]:
df2 = pd.read_csv('/Users/davidfernandez/Desktop/clean/scrapped/2016-2017.csv') 

df2 = df2[df2['Link perfil'] != 'Link no existente']

df2 = df2.reset_index()

df2 = df2.drop('index', axis=1)

In [906]:
df2["Datos escrapeados"] = df2["Link perfil"].apply(scrape_linkedin_profile2)


In [907]:
valor_a_buscar = {'code': 404, 'description': 'Person profile does not exist', 'name': 'Not Found'}


filas_filtradas = df2.loc[df2['Datos escrapeados'] == valor_a_buscar]

# We identify those students whose data we couldn't scrap

indices_a_eliminar = filas_filtradas.index

df2 = df2.drop(indices_a_eliminar)

# And we take them out 

In [916]:
copi2 = df2.copy()

In [918]:
copi2 = copi2.reset_index()
copi2 = copi2.drop('index', axis=1)

normalized_data = pd.json_normalize(copi2['Datos escrapeados'])

copi2 = pd.concat([copi2, normalized_data], axis=1)

columnas_a_eliminar = ['Nombre', 'Primer apellido', 'Segundo apellido', 'Email universitario', 'Email personal', 'Datos escrapeados', 'public_identifier', 'profile_pic_url', 'first_name', 'last_name', 'full_name', 'headline', 'country', 'languages', 'education', 'occupation', 'connections', 'country_full_name', 'follower_count', 'summary', 'state', 'accomplishment_honors_awards', 'accomplishment_courses', 'accomplishment_projects', 'groups', 'accomplishment_publications', 'certifications', 'accomplishment_organisations', 'accomplishment_test_scores']

copi2 = copi2.drop(columns=columnas_a_eliminar)

copi2 = copi2.explode('experiences')

copi2 = copi2.reset_index()
copi2 = copi2.drop('index', axis=1)

normalized_data1 = pd.json_normalize(copi2['experiences'])

copi2 = pd.concat([copi2, normalized_data1], axis=1)

cols_a_eliminar = ['description', 'logo_url', 'ends_at', 'starts_at', 'experiences']

copi2 = copi2.drop(cols_a_eliminar, axis=1)


In [920]:
copi2['ends_at.day'].fillna('actualidad', inplace=True)
copi2['ends_at.month'].fillna('actualidad', inplace=True)
copi2['ends_at.year'].fillna('actualidad', inplace=True)

copi2.dropna(subset=['starts_at.day', 'starts_at.month', 'starts_at.year'], inplace=True)

copi2['Fecha inicio'] = copi2.apply(lambda x: pd.to_datetime(f"{int(x['starts_at.year'])}-{int(x['starts_at.month'])}-01"), axis=1)
copi2['Fecha fin'] = copi2.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else pd.to_datetime(f"{int(x['ends_at.year'])}-{int(x['ends_at.month'])}-01"), axis=1)

copi2['Duración (meses)'] = copi2.apply(lambda x: (datetime.datetime.now() - x['Fecha inicio']).days // 30 if x['ends_at.year'] == 'actualidad' else (x['Fecha fin'] - x['Fecha inicio']).days // 30, axis=1)

copi2['Fecha fin'] = copi2.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else x['Fecha fin'], axis=1)

copi2.drop(['location', 'starts_at.year', 'starts_at.day', 'starts_at.month', 'ends_at.day', 'ends_at.month', 'ends_at.year'], axis=1, inplace=True)

copi2.rename(columns={'company': 'Empresa', 'title': 'Puesto', 'city': 'Ciudad'}, inplace=True)

copi2 = copi2.reset_index()
copi2 = copi2.drop('index', axis=1)





In [923]:
copi2["company_linkedin_profile_url"] = copi2["company_linkedin_profile_url"].apply(scrape_linkedin_company_data)


Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de No

Error al obtener datos de https://www.linkedin.com/company/dealz-espa%C3%B1a/ con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de https://es.linkedin.com/company/el-espa%C3%B1ol con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda 

Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/1207/ con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda op

In [931]:
copii2 = copi2.copy()

In [933]:
normalized_data1 = pd.json_normalize(copii2['company_linkedin_profile_url']) 


copii2 = pd.concat([copii2, normalized_data1], axis=1) 

In [935]:
to_drop1 = ['linkedin_internal_id', 'specialities', 'company_linkedin_profile_url', 'company_type', 'description', 'website', 'company_size', 'company_size_on_linkedin', 'founded_year', 'locations', 'name', 'tagline', 'universal_name_id', 'profile_pic_url', 'background_cover_image_url', 'search_id', 'similar_companies', 'affiliated_companies', 'updates', 'follower_count', 'acquisitions', 'exit_data', 'extra', 'funding_data', 'categories', 'hq.country', 'hq.city', 'hq.postal_code', 'hq.line_1', 'hq.is_hq', 'hq.state', 'hq']


copii2.drop(to_drop1, axis=1, inplace=True)

In [939]:
copii2.rename(columns={'industry': 'Sector'}, inplace=True)

In [940]:
new_order = ['Grado', 'Nombre completo', 'Link perfil', 'Puesto', 'Empresa', 'Sector', 'Ciudad', 'Fecha inicio', 'Fecha fin', 'Duración (meses)']

copii2 = copii2[new_order]

In [942]:
copii2.to_csv('/Users/davidfernandez/Desktop/clean/experience/2016-2017.csv', index=False) 


## Third dataframe: 2017-2018 class

In [602]:
df3 = pd.read_csv('/Users/davidfernandez/Desktop/clean/scrapped/2017-2018.csv') 

df3 = df3[df3['Link perfil'] != 'Link no existente']

df3 = df3.reset_index()

df3 = df3.drop('index', axis=1)

In [604]:
df3["Datos escrapeados"] = df3["Link perfil"].apply(scrape_linkedin_profile2)


In [606]:
valor_a_buscar = {'code': 404, 'description': 'Person profile does not exist', 'name': 'Not Found'}


filas_filtradas = df3.loc[df3['Datos escrapeados'] == valor_a_buscar]

# We identify those students whose data we couldn't scrap

indices_a_eliminar = filas_filtradas.index

df3 = df3.drop(indices_a_eliminar)

# And we take them out 

In [948]:
copi3 = df3.copy()

In [949]:
copi3 = copi3.reset_index()
copi3 = copi3.drop('index', axis=1)

normalized_data = pd.json_normalize(copi3['Datos escrapeados'])

copi3 = pd.concat([copi3, normalized_data], axis=1)

columnas_a_eliminar = ['Email universitario', 'Email personal', 'Datos escrapeados', 'public_identifier', 'profile_pic_url', 'first_name', 'last_name', 'full_name', 'headline', 'country', 'languages', 'education', 'occupation', 'connections', 'country_full_name', 'follower_count', 'summary', 'state', 'accomplishment_honors_awards', 'accomplishment_courses', 'accomplishment_projects', 'groups', 'accomplishment_publications', 'certifications', 'accomplishment_organisations', 'accomplishment_test_scores']

copi3 = copi3.drop(columns=columnas_a_eliminar)

copi3 = copi3.explode('experiences')

copi3 = copi3.reset_index()
copi3 = copi3.drop('index', axis=1)

normalized_data1 = pd.json_normalize(copi3['experiences'])

copi3 = pd.concat([copi3, normalized_data1], axis=1)

cols_a_eliminar = ['description', 'logo_url', 'ends_at', 'starts_at', 'experiences']

copi3 = copi3.drop(cols_a_eliminar, axis=1)


In [950]:
copi3['ends_at.day'].fillna('actualidad', inplace=True)
copi3['ends_at.month'].fillna('actualidad', inplace=True)
copi3['ends_at.year'].fillna('actualidad', inplace=True)

copi3.dropna(subset=['starts_at.day', 'starts_at.month', 'starts_at.year'], inplace=True)

copi3['Fecha inicio'] = copi3.apply(lambda x: pd.to_datetime(f"{int(x['starts_at.year'])}-{int(x['starts_at.month'])}-01"), axis=1)
copi3['Fecha fin'] = copi3.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else pd.to_datetime(f"{int(x['ends_at.year'])}-{int(x['ends_at.month'])}-01"), axis=1)

copi3['Duración (meses)'] = copi3.apply(lambda x: (datetime.datetime.now() - x['Fecha inicio']).days // 30 if x['ends_at.year'] == 'actualidad' else (x['Fecha fin'] - x['Fecha inicio']).days // 30, axis=1)

copi3['Fecha fin'] = copi3.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else x['Fecha fin'], axis=1)

copi3.drop(['location', 'starts_at.year', 'starts_at.day', 'starts_at.month', 'ends_at.day', 'ends_at.month', 'ends_at.year'], axis=1, inplace=True)

copi3.rename(columns={'company': 'Empresa', 'title': 'Puesto', 'city': 'Ciudad'}, inplace=True)

copi3 = copi3.reset_index()
copi3 = copi3.drop('index', axis=1)




In [952]:
copi3["Datos empresa"] = copi3["company_linkedin_profile_url"].apply(scrape_linkedin_company_data)


Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de https://es.linkedin.com/company/non-stop-people-espa%C3%B1a con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de https://es.linkedin.com/company/popeyes-espa%%C3%%B1a con la segunda opción.
Error al obtener datos de https://es.linkedin.com/company/caf%%C3%%A9-leather-supply con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de https://es.linkedin.com/company/lor%C3%A9al con la segunda opción.
Error al obtener datos de https://www.linkedin.

Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/lor%C3%A9al/ con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la seg

Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de No

In [953]:
copii3 = copi3.copy()

In [955]:
normalized_data1 = pd.json_normalize(copii3['Datos empresa']) 

copii3 = pd.concat([copii3, normalized_data1], axis=1) 

In [956]:
to_drop1 = ['linkedin_internal_id', 'specialities', 'Datos empresa', 'company_type', 'description', 'website', 'company_size', 'company_size_on_linkedin', 'founded_year', 'locations', 'name', 'tagline', 'universal_name_id', 'profile_pic_url', 'background_cover_image_url', 'search_id', 'similar_companies', 'affiliated_companies', 'updates', 'follower_count', 'acquisitions', 'exit_data', 'extra', 'funding_data', 'categories', 'hq.country', 'hq.city', 'hq.postal_code', 'hq.line_1', 'hq.is_hq', 'hq.state', 'hq']


copii3.drop(to_drop1, axis=1, inplace=True)

In [958]:
copii3.rename(columns={'industry': 'Sector'}, inplace=True)

new_order = ['Grado', 'Nombre completo', 'Link perfil', 'Puesto', 'Empresa', 'Sector', 'Ciudad', 'Fecha inicio', 'Fecha fin', 'Duración (meses)']

copii3 = copii3[new_order]

In [960]:
copii3.to_csv('/Users/davidfernandez/Desktop/clean/experience/2017-2018.csv', index=False)

## Fifth dataframe: 2019-2020 class

In [626]:
df5 = pd.read_csv('/Users/davidfernandez/Desktop/clean/scrapped/2019-2020.csv')

df5 = df5[df5['Link perfil'] != 'Link no existente']

df5 = df5.reset_index()

df5 = df5.drop('index', axis=1)

In [627]:
df5["Datos escrapeados"] = df5["Link perfil"].apply(scrape_linkedin_profile2)

In [629]:
valor_a_buscar = {'code': 404, 'description': 'Person profile does not exist', 'name': 'Not Found'}


filas_filtradas = df5.loc[df5['Datos escrapeados'] == valor_a_buscar]

# We identify those students whose data we couldn't scrap

indices_a_eliminar = filas_filtradas.index

df5 = df5.drop(indices_a_eliminar)

# And we take them out 


In [972]:
copi5 = df5.copy()

In [973]:
copi5 = copi5.reset_index()
copi5 = copi5.drop('index', axis=1)

normalized_data = pd.json_normalize(copi5['Datos escrapeados'])

copi5 = pd.concat([copi5, normalized_data], axis=1)

columnas_a_eliminar = ['Email universitario', 'Email personal', 'Datos escrapeados', 'public_identifier', 'profile_pic_url', 'first_name', 'last_name', 'full_name', 'headline', 'country', 'languages', 'education', 'occupation', 'connections', 'country_full_name', 'follower_count', 'summary', 'state', 'accomplishment_honors_awards', 'accomplishment_courses', 'accomplishment_projects', 'groups', 'accomplishment_publications', 'certifications', 'accomplishment_organisations', 'accomplishment_test_scores']

copi5 = copi5.drop(columns=columnas_a_eliminar)

copi5 = copi5.explode('experiences')

copi5 = copi5.reset_index()
copi5 = copi5.drop('index', axis=1)

normalized_data1 = pd.json_normalize(copi5['experiences'])

copi5 = pd.concat([copi5, normalized_data1], axis=1)

cols_a_eliminar = ['description', 'logo_url', 'ends_at', 'starts_at', 'experiences']

copi5 = copi5.drop(cols_a_eliminar, axis=1)



In [974]:
copi5['ends_at.day'].fillna('actualidad', inplace=True)
copi5['ends_at.month'].fillna('actualidad', inplace=True)
copi5['ends_at.year'].fillna('actualidad', inplace=True)

copi5.dropna(subset=['starts_at.day', 'starts_at.month', 'starts_at.year'], inplace=True)

copi5['Fecha inicio'] = copi5.apply(lambda x: pd.to_datetime(f"{int(x['starts_at.year'])}-{int(x['starts_at.month'])}-01"), axis=1)
copi5['Fecha fin'] = copi5.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else pd.to_datetime(f"{int(x['ends_at.year'])}-{int(x['ends_at.month'])}-01"), axis=1)

copi5['Duración (meses)'] = copi5.apply(lambda x: (datetime.datetime.now() - x['Fecha inicio']).days // 30 if x['ends_at.year'] == 'actualidad' else (x['Fecha fin'] - x['Fecha inicio']).days // 30, axis=1)

copi5['Fecha fin'] = copi5.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else x['Fecha fin'], axis=1)

copi5.drop(['location', 'starts_at.year', 'starts_at.day', 'starts_at.month', 'ends_at.day', 'ends_at.month', 'ends_at.year'], axis=1, inplace=True)

copi5.rename(columns={'company': 'Empresa', 'title': 'Puesto', 'city': 'Ciudad'}, inplace=True)

copi5 = copi5.reset_index()
copi5 = copi5.drop('index', axis=1)


In [976]:
copi5["Datos empresa"] = copi5["company_linkedin_profile_url"].apply(scrape_linkedin_company_data)


Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/fusi%C3%B3n-coartada/ con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/lor%C3%A9al/ con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de https://es.linkedin.com/company/fundacion-universitaria-san-pablo-ceu con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con

In [978]:
copii5 = copi5.copy()

normalized_data1 = pd.json_normalize(copii5['Datos empresa']) 

copii5 = pd.concat([copii5, normalized_data1], axis=1) 

to_drop1 = ['linkedin_internal_id', 'specialities', 'Datos empresa', 'company_type', 'description', 'website', 'company_size', 'company_size_on_linkedin', 'founded_year', 'locations', 'name', 'tagline', 'universal_name_id', 'profile_pic_url', 'background_cover_image_url', 'search_id', 'similar_companies', 'affiliated_companies', 'updates', 'follower_count', 'acquisitions', 'exit_data', 'extra', 'funding_data', 'categories', 'hq.country', 'hq.city', 'hq.postal_code', 'hq.line_1', 'hq.is_hq', 'hq.state', 'hq']

copii5.drop(to_drop1, axis=1, inplace=True)

copii5.rename(columns={'industry': 'Sector'}, inplace=True)

new_order = ['Grado', 'Nombre completo', 'Link perfil', 'Puesto', 'Empresa', 'Sector', 'Ciudad', 'Fecha inicio', 'Fecha fin', 'Duración (meses)']

copii5 = copii5[new_order]


In [980]:
copii5.to_csv('/Users/davidfernandez/Desktop/clean/experience/2019-2020.csv', index=False)

## Sixth dataframe: 2020-2021 class

In [635]:
df6 = pd.read_csv('/Users/davidfernandez/Desktop/clean/scrapped/2020-2021.csv')

df6 = df6[df6['Link perfil'] != 'Link no existente']

df6 = df6.reset_index()

df6 = df6.drop('index', axis=1)

In [637]:
df6["Datos escrapeados"] = df6["Link perfil"].apply(scrape_linkedin_profile2)

In [639]:
valor_a_buscar = {'code': 404, 'description': 'Person profile does not exist', 'name': 'Not Found'}


filas_filtradas = df6.loc[df6['Datos escrapeados'] == valor_a_buscar]

# We identify those students whose data we couldn't scrap

indices_a_eliminar = filas_filtradas.index

df6 = df6.drop(indices_a_eliminar)

# And we take them out 


In [990]:
copi6 = df6.copy()

In [991]:
copi6 = copi6.reset_index()
copi6 = copi6.drop('index', axis=1)

normalized_data = pd.json_normalize(copi6['Datos escrapeados'])

copi6 = pd.concat([copi6, normalized_data], axis=1)

columnas_a_eliminar = ['Email universitario', 'Email personal', 'Datos escrapeados', 'public_identifier', 'profile_pic_url', 'first_name', 'last_name', 'full_name', 'headline', 'country', 'languages', 'education', 'occupation', 'connections', 'country_full_name', 'follower_count', 'summary', 'state', 'accomplishment_honors_awards', 'accomplishment_courses', 'accomplishment_projects', 'groups', 'accomplishment_publications', 'certifications', 'accomplishment_organisations', 'accomplishment_test_scores']

copi6 = copi6.drop(columns=columnas_a_eliminar)

copi6 = copi6.explode('experiences')

copi6 = copi6.reset_index()
copi6 = copi6.drop('index', axis=1)

normalized_data1 = pd.json_normalize(copi6['experiences'])

copi6 = pd.concat([copi6, normalized_data1], axis=1)

cols_a_eliminar = ['description', 'logo_url', 'ends_at', 'starts_at', 'experiences']

copi6 = copi6.drop(cols_a_eliminar, axis=1)


In [992]:
copi6['ends_at.day'].fillna('actualidad', inplace=True)
copi6['ends_at.month'].fillna('actualidad', inplace=True)
copi6['ends_at.year'].fillna('actualidad', inplace=True)

copi6.dropna(subset=['starts_at.day', 'starts_at.month', 'starts_at.year'], inplace=True)

copi6['Fecha inicio'] = copi6.apply(lambda x: pd.to_datetime(f"{int(x['starts_at.year'])}-{int(x['starts_at.month'])}-01"), axis=1)
copi6['Fecha fin'] = copi6.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else pd.to_datetime(f"{int(x['ends_at.year'])}-{int(x['ends_at.month'])}-01"), axis=1)

copi6['Duración (meses)'] = copi6.apply(lambda x: (datetime.datetime.now() - x['Fecha inicio']).days // 30 if x['ends_at.year'] == 'actualidad' else (x['Fecha fin'] - x['Fecha inicio']).days // 30, axis=1)

copi6['Fecha fin'] = copi6.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else x['Fecha fin'], axis=1)

copi6.drop(['location', 'starts_at.year', 'starts_at.day', 'starts_at.month', 'ends_at.day', 'ends_at.month', 'ends_at.year'], axis=1, inplace=True)

copi6.rename(columns={'company': 'Empresa', 'title': 'Puesto', 'city': 'Ciudad'}, inplace=True)

copi6 = copi6.reset_index()
copi6 = copi6.drop('index', axis=1)


In [994]:
copi6["Datos empresa"] = copi6["company_linkedin_profile_url"].apply(scrape_linkedin_company_data)


Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/psic%C3%B3logos-majadahonda/ con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/bamb%C3%BA-producciones/ con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de 

Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.


In [995]:
normalized_data1 = pd.json_normalize(copi6['Datos empresa']) 

copi6 = pd.concat([copi6, normalized_data1], axis=1) 

to_drop1 = ['linkedin_internal_id', 'specialities', 'Datos empresa', 'company_type', 'description', 'website', 'company_size', 'company_size_on_linkedin', 'founded_year', 'locations', 'name', 'tagline', 'universal_name_id', 'profile_pic_url', 'background_cover_image_url', 'search_id', 'similar_companies', 'affiliated_companies', 'updates', 'follower_count', 'acquisitions', 'exit_data', 'extra', 'funding_data', 'categories', 'hq.country', 'hq.city', 'hq.postal_code', 'hq.line_1', 'hq.is_hq', 'hq.state', 'hq']

copi6.drop(to_drop1, axis=1, inplace=True)

copi6.rename(columns={'industry': 'Sector'}, inplace=True)

new_order = ['Grado', 'Nombre completo', 'Link perfil', 'Puesto', 'Empresa', 'Sector', 'Ciudad', 'Fecha inicio', 'Fecha fin', 'Duración (meses)']

copi6 = copi6[new_order]

In [997]:
copi6.to_csv('/Users/davidfernandez/Desktop/clean/experience/2020-2021.csv', index=False)

## Seventh dataframe: 2021-2022 class

In [645]:
df7 = pd.read_csv('/Users/davidfernandez/Desktop/clean/scrapped/2021-2022.csv')

df7 = df7[df7['Link perfil'] != 'Link no existente']

df7 = df7.reset_index()

df7 = df7.drop('index', axis=1)

In [647]:
df7["Datos escrapeados"] = df7["Link perfil"].apply(scrape_linkedin_profile2)

In [649]:
valor_a_buscar = {'code': 404, 'description': 'Person profile does not exist', 'name': 'Not Found'}


filas_filtradas = df7.loc[df7['Datos escrapeados'] == valor_a_buscar]

# We identify those students whose data we couldn't scrap

indices_a_eliminar = filas_filtradas.index

df7 = df7.drop(indices_a_eliminar)

# And we take them out 

In [1001]:
copi7 = df7.copy()

In [1002]:
copi7 = copi7.reset_index()
copi7 = copi7.drop('index', axis=1)

normalized_data = pd.json_normalize(copi7['Datos escrapeados'])

copi7 = pd.concat([copi7, normalized_data], axis=1)

columnas_a_eliminar = ['Email universitario', 'Email personal', 'Datos escrapeados', 'public_identifier', 'profile_pic_url', 'first_name', 'last_name', 'full_name', 'headline', 'country', 'languages', 'education', 'occupation', 'connections', 'country_full_name', 'follower_count', 'summary', 'state', 'accomplishment_honors_awards', 'accomplishment_courses', 'accomplishment_projects', 'groups', 'accomplishment_publications', 'certifications', 'accomplishment_organisations', 'accomplishment_test_scores']

copi7 = copi7.drop(columns=columnas_a_eliminar)

copi7 = copi7.explode('experiences')

copi7 = copi7.reset_index()
copi7 = copi7.drop('index', axis=1)

normalized_data1 = pd.json_normalize(copi7['experiences'])

copi7 = pd.concat([copi7, normalized_data1], axis=1)

cols_a_eliminar = ['description', 'logo_url', 'ends_at', 'starts_at', 'experiences']

copi7 = copi7.drop(cols_a_eliminar, axis=1)


In [1003]:
copi7['ends_at.day'].fillna('actualidad', inplace=True)
copi7['ends_at.month'].fillna('actualidad', inplace=True)
copi7['ends_at.year'].fillna('actualidad', inplace=True)

copi7.dropna(subset=['starts_at.day', 'starts_at.month', 'starts_at.year'], inplace=True)

copi7['Fecha inicio'] = copi7.apply(lambda x: pd.to_datetime(f"{int(x['starts_at.year'])}-{int(x['starts_at.month'])}-01"), axis=1)
copi7['Fecha fin'] = copi7.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else pd.to_datetime(f"{int(x['ends_at.year'])}-{int(x['ends_at.month'])}-01"), axis=1)

copi7['Duración (meses)'] = copi7.apply(lambda x: (datetime.datetime.now() - x['Fecha inicio']).days // 30 if x['ends_at.year'] == 'actualidad' else (x['Fecha fin'] - x['Fecha inicio']).days // 30, axis=1)

copi7['Fecha fin'] = copi7.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else x['Fecha fin'], axis=1)

copi7.drop(['location', 'starts_at.year', 'starts_at.day', 'starts_at.month', 'ends_at.day', 'ends_at.month', 'ends_at.year'], axis=1, inplace=True)

copi7.rename(columns={'company': 'Empresa', 'title': 'Puesto', 'city': 'Ciudad'}, inplace=True)

copi7 = copi7.reset_index()
copi7 = copi7.drop('index', axis=1)


In [1005]:
copi7["Datos empresa"] = copi7["company_linkedin_profile_url"].apply(scrape_linkedin_company_data)


Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/psic%C3%B3logos-majadahonda/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/10660436/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/3862919/ con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Erro

Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/keep-doing-producci%C3%B3n/ con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de 

In [1006]:
normalized_data1 = pd.json_normalize(copi7['Datos empresa']) 

copi7 = pd.concat([copi7, normalized_data1], axis=1) 

to_drop1 = ['linkedin_internal_id', 'specialities', 'Datos empresa', 'company_type', 'description', 'website', 'company_size', 'company_size_on_linkedin', 'founded_year', 'locations', 'name', 'tagline', 'universal_name_id', 'profile_pic_url', 'background_cover_image_url', 'search_id', 'similar_companies', 'affiliated_companies', 'updates', 'follower_count', 'acquisitions', 'exit_data', 'extra', 'funding_data', 'categories', 'hq.country', 'hq.city', 'hq.postal_code', 'hq.line_1', 'hq.is_hq', 'hq.state', 'hq']

copi7.drop(to_drop1, axis=1, inplace=True)

copi7.rename(columns={'industry': 'Sector'}, inplace=True)

new_order = ['Grado', 'Nombre completo', 'Link perfil', 'Puesto', 'Empresa', 'Sector', 'Ciudad', 'Fecha inicio', 'Fecha fin', 'Duración (meses)']

copi7 = copi7[new_order]

In [1008]:
copi7.to_csv('/Users/davidfernandez/Desktop/clean/experience/2021-2022.csv', index=False)

## Eight dataframe: 2022-2023 class

In [661]:
df8 = pd.read_csv('/Users/davidfernandez/Desktop/clean/scrapped/2022-2023.csv')

df8 = df8[df8['Link perfil'] != 'Link no existente']

df8 = df8.reset_index()

df8 = df8.drop('index', axis=1)

In [664]:
df8["Datos escrapeados"] = df8["Link perfil"].apply(scrape_linkedin_profile2)

In [666]:
valor_a_buscar = {'code': 404, 'description': 'Person profile does not exist', 'name': 'Not Found'}


filas_filtradas = df8.loc[df8['Datos escrapeados'] == valor_a_buscar]

# We identify those students whose data we couldn't scrap

indices_a_eliminar = filas_filtradas.index

df8 = df8.drop(indices_a_eliminar)

# And we take them out 

In [1011]:
copi8 = df8.copy()

In [1012]:
copi8 = copi8.reset_index()
copi8 = copi8.drop('index', axis=1)

normalized_data = pd.json_normalize(copi8['Datos escrapeados'])

copi8 = pd.concat([copi8, normalized_data], axis=1)

columnas_a_eliminar = ['Email universitario', 'Datos escrapeados', 'public_identifier', 'profile_pic_url', 'first_name', 'last_name', 'full_name', 'headline', 'country', 'languages', 'education', 'occupation', 'connections', 'country_full_name', 'follower_count', 'summary', 'state', 'accomplishment_honors_awards', 'accomplishment_courses', 'accomplishment_projects', 'groups', 'accomplishment_publications', 'certifications', 'accomplishment_organisations', 'accomplishment_test_scores']

copi8 = copi8.drop(columns=columnas_a_eliminar)

copi8 = copi8.explode('experiences')

copi8 = copi8.reset_index()
copi8 = copi8.drop('index', axis=1)

normalized_data1 = pd.json_normalize(copi8['experiences'])

copi8 = pd.concat([copi8, normalized_data1], axis=1)

cols_a_eliminar = ['description', 'logo_url', 'ends_at', 'starts_at', 'experiences']

copi8 = copi8.drop(cols_a_eliminar, axis=1)

In [1013]:
copi8['ends_at.day'].fillna('actualidad', inplace=True)
copi8['ends_at.month'].fillna('actualidad', inplace=True)
copi8['ends_at.year'].fillna('actualidad', inplace=True)

copi8.dropna(subset=['starts_at.day', 'starts_at.month', 'starts_at.year'], inplace=True)

copi8['Fecha inicio'] = copi8.apply(lambda x: pd.to_datetime(f"{int(x['starts_at.year'])}-{int(x['starts_at.month'])}-01"), axis=1)
copi8['Fecha fin'] = copi8.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else pd.to_datetime(f"{int(x['ends_at.year'])}-{int(x['ends_at.month'])}-01"), axis=1)

copi8['Duración (meses)'] = copi8.apply(lambda x: (datetime.datetime.now() - x['Fecha inicio']).days // 30 if x['ends_at.year'] == 'actualidad' else (x['Fecha fin'] - x['Fecha inicio']).days // 30, axis=1)

copi8['Fecha fin'] = copi8.apply(lambda x: 'actualidad' if x['ends_at.year'] == 'actualidad' else x['Fecha fin'], axis=1)

copi8.drop(['location', 'starts_at.year', 'starts_at.day', 'starts_at.month', 'ends_at.day', 'ends_at.month', 'ends_at.year'], axis=1, inplace=True)

copi8.rename(columns={'company': 'Empresa', 'title': 'Puesto', 'city': 'Ciudad'}, inplace=True)

copi8 = copi8.reset_index()
copi8 = copi8.drop('index', axis=1)

In [1015]:
copi8["Datos empresa"] = copi8["company_linkedin_profile_url"].apply(scrape_linkedin_company_data)


Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/79893405/ con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/real-f%C3%A1brica-de-tapices/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/fundaci%C3%B3n-universitaria-san-pablo-unisanpablo/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/fundaci%C3%B3n-universitaria-san-pablo-unisanpablo/ con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segu

Error al obtener datos de None con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/40132126/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/73129443/ con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/15092653/ con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de None con la segunda opción.
Error al obtener datos de https://www.linkedin.com/company/hospital-central-de-la-defensa-g%C3%B3mez-ulla/ con la segunda opción.


In [1016]:
normalized_data1 = pd.json_normalize(copi8['Datos empresa']) 

copi8 = pd.concat([copi8, normalized_data1], axis=1) 

to_drop1 = ['linkedin_internal_id', 'specialities', 'Datos empresa', 'company_type', 'description', 'website', 'company_size', 'company_size_on_linkedin', 'founded_year', 'locations', 'name', 'tagline', 'universal_name_id', 'profile_pic_url', 'background_cover_image_url', 'search_id', 'similar_companies', 'affiliated_companies', 'updates', 'follower_count', 'acquisitions', 'exit_data', 'extra', 'funding_data', 'categories', 'hq.country', 'hq.city', 'hq.postal_code', 'hq.line_1', 'hq.is_hq', 'hq.state', 'hq']

copi8.drop(to_drop1, axis=1, inplace=True)

copi8.rename(columns={'industry': 'Sector'}, inplace=True)

new_order = ['Grado', 'Nombre completo', 'Link perfil', 'Puesto', 'Empresa', 'Sector', 'Ciudad', 'Fecha inicio', 'Fecha fin', 'Duración (meses)']

copi8 = copi8[new_order]

In [1018]:
copi8.to_csv('/Users/davidfernandez/Desktop/clean/experience/2022-2023.csv', index=False)

# We add a final column with graduation date and concatenate every dataframe

In [1033]:
df11 = df1.copy()

In [1041]:
df11['Fecha graduación'] = pd.to_datetime('2016-06-01')
copii2['Fecha graduación'] = pd.to_datetime('2017-06-01')
copii3['Fecha graduación'] = pd.to_datetime('2018-06-01')
copii5['Fecha graduación'] = pd.to_datetime('2020-06-01')
copi6['Fecha graduación'] = pd.to_datetime('2021-06-01')
copi7['Fecha graduación'] = pd.to_datetime('2022-06-01')
copi8['Fecha graduación'] = pd.to_datetime('2023-06-01')



/var/folders/y6/y46jwd8j5nn1qp81l6_znfkh0000gn/T/ipykernel_76198/891368230.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  copii2['Fecha graduación'] = pd.to_datetime('2017-06-01')


In [1049]:
lista_dataframes = [df11, copii2, copii3, copii5, copi6, copi7, copi8]  

finaldf = pd.concat(lista_dataframes, ignore_index=True)


In [1051]:
finaldf.to_csv('/Users/davidfernandez/Desktop/clean/experience/final.csv', index=False) 
